In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm
import re
import collections
import statsmodels.api as sm

In [3]:
requests.get('https://carvago.com/robots.txt')

<Response [200]>

In [4]:
print(requests.get('https://carvago.com/robots.txt').text)

User-agent: *
Disallow: 
Sitemap: https://carvago.com/sitemap.xml
Sitemap: https://carvago.com/sitemap-listed-cars.xml
Sitemap: https://carvago.com/sitemap-makes-models.xml


In [5]:
def getSoup(link):
    sleep(0.1) #to be kind to the website
    r = requests.get(link)
    r.encoding = 'UTF-8'
    return BeautifulSoup(r.text,'lxml')

In [6]:
soup = getSoup('https://carvago.com/cs/auta/skoda')

In [7]:
divs = soup.findAll('div', {'class':'css-1dapwlk e1oahio83'})

In [9]:
def getAllLinks(link):
    soup = getSoup(link)
    divs = soup.findAll('div', {'class':'css-1dapwlk e1oahio83'})
    return ['https://carvago.com' + div.find('a')['href'] for div in divs]

first_page = getAllLinks('https://carvago.com/cs/auta/skoda')
first_page

['https://carvago.com/cs/auto/43025505/skoda-fabia-1-2-tsi-66-kw',
 'https://carvago.com/cs/auto/48304837/skoda-rapid-70-kw',
 'https://carvago.com/cs/auto/47919164/skoda-rapid-1-2-tsi-green-tec-style-77-kw',
 'https://carvago.com/cs/auto/48149917/skoda-superb-110-kw',
 'https://carvago.com/cs/auto/48285494/skoda-scala-110-kw',
 'https://carvago.com/cs/auto/47035228/skoda-superb-147-kw',
 'https://carvago.com/cs/auto/48392384/skoda-citigo-1-0-mpi-44-kw',
 'https://carvago.com/cs/auto/48000196/skoda-citigo-1-0-mpi-tec-44-kw',
 'https://carvago.com/cs/auto/48318401/skoda-kodiaq-2-0-tsi-act-4x4-dsg-140-kw',
 'https://carvago.com/cs/auto/46729957/skoda-kodiaq-180-kw',
 'https://carvago.com/cs/auto/47949633/skoda-enyaq-iv-80-150-kw',
 'https://carvago.com/cs/auto/47801782/skoda-karoq-1-0-tsi-ambition-85-kw',
 'https://carvago.com/cs/auto/44593683/skoda-citigo-1-0-mpi-tec-44-kw',
 'https://carvago.com/cs/auto/48417416/skoda-citigo-1-0-mpi-tec-clever-44-kw',
 'https://carvago.com/cs/auto/4376

In [11]:
print([item for item, count in collections.Counter(first_page).items() if count > 1]) #no duplicates

[]


In [47]:
urls=[]
urls

[]

In [48]:
#this code will scrape (n-1) pages but one twice
for i in range (12):
    urls.append(getAllLinks('https://carvago.com/cs/auta/skoda?page='+str(i)))

In [49]:
urls = sum(urls,[])
urls

['https://carvago.com/cs/auto/43025505/skoda-fabia-1-2-tsi-66-kw',
 'https://carvago.com/cs/auto/48304837/skoda-rapid-70-kw',
 'https://carvago.com/cs/auto/47919164/skoda-rapid-1-2-tsi-green-tec-style-77-kw',
 'https://carvago.com/cs/auto/48149917/skoda-superb-110-kw',
 'https://carvago.com/cs/auto/48285494/skoda-scala-110-kw',
 'https://carvago.com/cs/auto/47035228/skoda-superb-147-kw',
 'https://carvago.com/cs/auto/48392384/skoda-citigo-1-0-mpi-44-kw',
 'https://carvago.com/cs/auto/48000196/skoda-citigo-1-0-mpi-tec-44-kw',
 'https://carvago.com/cs/auto/48318401/skoda-kodiaq-2-0-tsi-act-4x4-dsg-140-kw',
 'https://carvago.com/cs/auto/46729957/skoda-kodiaq-180-kw',
 'https://carvago.com/cs/auto/47949633/skoda-enyaq-iv-80-150-kw',
 'https://carvago.com/cs/auto/47801782/skoda-karoq-1-0-tsi-ambition-85-kw',
 'https://carvago.com/cs/auto/44593683/skoda-citigo-1-0-mpi-tec-44-kw',
 'https://carvago.com/cs/auto/48417416/skoda-citigo-1-0-mpi-tec-clever-44-kw',
 'https://carvago.com/cs/auto/4376

In [50]:
print([item for item, count in collections.Counter(urls).items() if count > 1])

['https://carvago.com/cs/auto/43025505/skoda-fabia-1-2-tsi-66-kw', 'https://carvago.com/cs/auto/48304837/skoda-rapid-70-kw', 'https://carvago.com/cs/auto/47919164/skoda-rapid-1-2-tsi-green-tec-style-77-kw', 'https://carvago.com/cs/auto/48149917/skoda-superb-110-kw', 'https://carvago.com/cs/auto/48285494/skoda-scala-110-kw', 'https://carvago.com/cs/auto/47035228/skoda-superb-147-kw', 'https://carvago.com/cs/auto/48392384/skoda-citigo-1-0-mpi-44-kw', 'https://carvago.com/cs/auto/48000196/skoda-citigo-1-0-mpi-tec-44-kw', 'https://carvago.com/cs/auto/48318401/skoda-kodiaq-2-0-tsi-act-4x4-dsg-140-kw', 'https://carvago.com/cs/auto/46729957/skoda-kodiaq-180-kw', 'https://carvago.com/cs/auto/47949633/skoda-enyaq-iv-80-150-kw', 'https://carvago.com/cs/auto/47801782/skoda-karoq-1-0-tsi-ambition-85-kw', 'https://carvago.com/cs/auto/44593683/skoda-citigo-1-0-mpi-tec-44-kw', 'https://carvago.com/cs/auto/48417416/skoda-citigo-1-0-mpi-tec-clever-44-kw', 'https://carvago.com/cs/auto/43761533/skoda-eny

In [51]:
urls = set(urls)
print([item for item, count in collections.Counter(urls).items() if count > 1]) #no duplicates :)

[]


In [52]:
urls #only unique values

{'https://carvago.com/cs/auto/36838385/skoda-octavia-2-0-tdi-dpf-dsg-103-kw',
 'https://carvago.com/cs/auto/38365616/skoda-superb-2-0-tdi-dsg-110-kw',
 'https://carvago.com/cs/auto/40697335/skoda-superb-2-0-tsi-dsg-style-162-kw',
 'https://carvago.com/cs/auto/41313032/skoda-fabia-1-0-mpi-cool-edition-44-kw',
 'https://carvago.com/cs/auto/41527841/skoda-superb-2-0-tsi-4x4-dsg-l-k-200-kw',
 'https://carvago.com/cs/auto/41559628/skoda-superb-2-0-tsi-4x4-dsg-l-k-200-kw',
 'https://carvago.com/cs/auto/43025505/skoda-fabia-1-2-tsi-66-kw',
 'https://carvago.com/cs/auto/43157495/skoda-superb-1-4-tsi-act-dsg-110-kw',
 'https://carvago.com/cs/auto/43657584/skoda-karoq-1-5-tsi-act-dsg-110-kw',
 'https://carvago.com/cs/auto/43761533/skoda-enyaq-iv-80-150-kw',
 'https://carvago.com/cs/auto/43771444/skoda-enyaq-iv-80-150-kw',
 'https://carvago.com/cs/auto/43922343/skoda-superb-110-kw',
 'https://carvago.com/cs/auto/44108839/skoda-scala-110-kw',
 'https://carvago.com/cs/auto/44126106/skoda-scala-ambi

In [44]:
#creating object
class Car:
    def __init__(self,link):
        self.soup = getSoup(link)
        self.name = self.getName()
        self.model = self.getModel()
        self.price = self.getPrice()
        self.country = self.getCountry()
        self.mileage = self.getMileage()
        self.registration = self.getRegistration()
        self.transmission = self.getTransmission()
        self.power = self.getPower()
        self.fueltype = self.getFueltype()

    def getName(self):
        return self.soup.find('h1',{'class':'ea6i44d0 css-13i2d41 e5xkr5t0'}).text.strip()

    def getModel(self):
        return self.soup.find_all('div',{'class':'sc-dkPtRN kxDClK css-1b7garx ey3doxd4'})[1].text.strip()

    def getPrice(self):
        return self.soup.find('div',{'class':'css-1mbrvie e1hgzarh2'}).text.strip()

    def getCountry(self):
        return self.soup.find_all('div',{'class':'css-o5n6vn e51cp1p1'})[4].text.strip()

    def getMileage(self):
        return self.soup.find_all('div',{'class':'css-o5n6vn e51cp1p1'})[5].text.strip()

    def getRegistration(self):
        return self.soup.find_all('div',{'class':'css-o5n6vn e51cp1p1'})[6].text.strip()

    def getTransmission(self):
        return self.soup.find_all('div',{'class':'css-o5n6vn e51cp1p1'})[7].text.strip()

    def getPower(self):
        return self.soup.find_all('div',{'class':'css-o5n6vn e51cp1p1'})[8].text.strip()

    def getFueltype(self):
        return self.soup.find_all('div',{'class':'css-o5n6vn e51cp1p1'})[9].text.strip()

    def getmoreinfo(self):
        return pd.Series({
            'name':self.name,
            'model':self.model,
            'price':self.price,
            'country':self.country,
            'mileage':self.mileage,
            'registration':self.registration,
            'transmission':self.transmission,
            'power':self.power,
            'fueltype':self.fueltype
        })

In [53]:
cars = [Car(link) for link in urls]

In [54]:
[c.name for c in cars]

['Skoda Enyaq iV 60 132 kW',
 'Skoda Scala Style 110 kW',
 'Skoda Kodiaq 2.0 TSI ACT 4x4 DSG 140 kW',
 'Skoda Rapid 70 kW',
 'Skoda Scala 110 kW',
 'Skoda Octavia 1.2 TSI 63 kW',
 'Skoda Karoq 1.5 TSI ACT DSG 110 kW',
 'Skoda Rapid 70 kW',
 'Skoda Kodiaq 2.0 TSI ACT 4x4 DSG 140 kW',
 'Skoda Rapid 70 kW',
 'Skoda Rapid 66 kW',
 'Skoda Scala 1.5 TSI DSG Style 110 kW',
 'Skoda Karoq 1.5 TSI ACT DSG 110 kW',
 'Skoda Scala 110 kW',
 'Skoda Superb 147 kW',
 'Skoda Yeti 1.2 TSI Ambition 77 kW',
 'Skoda Scala Ambition 110 kW',
 'Skoda Enyaq iV 80 150 kW',
 'Skoda Fabia 1.0 TSI 70 kW',
 'Skoda Yeti 1.2 TSI 77 kW',
 'Skoda Superb 147 kW',
 'Skoda Karoq 1.0 TSI Ambition 85 kW',
 'Skoda Kodiaq 2.0 4x4 DSG RS 176 kW',
 'Skoda Citigo 1.0 MPI 44 kW',
 'Skoda Karoq 1.0 TSI Ambition 85 kW',
 'Skoda Scala 110 kW',
 'Skoda Octavia 2.0 TDI DSG Ambition 110 kW',
 'Skoda Kodiaq 180 kW',
 'Skoda Scala 110 kW',
 'Skoda Kodiaq 147 kW',
 'Skoda Octavia 2.0 TDI Green Tec Joy 110 kW',
 'Skoda Scala 110 kW',
 'Sko

In [55]:
dataframe = pd.DataFrame([c.getmoreinfo() for c in cars])
dataframe

,name,model,price,country,mileage,registration,transmission,power,fueltype
0,Skoda Enyaq iV 60 132 kW,Enyaq iV,986 990 Kč,Francie,2 000 km,12/2021,Automat,132 kW,Elektřina
1,Skoda Scala Style 110 kW,Scala,572 490 Kč,Německo,30 616 km,2/2020,Manuál,110 kW,Benzín
2,Skoda Kodiaq 2.0 TSI ACT 4x4 DSG 140 kW,Kodiaq,1 033 990 Kč,Německo,27 600 km,5/2021,Automat,140 kW,Benzín
3,Skoda Rapid 70 kW,Rapid,286 990 Kč,Polsko,25 147 km,1/2018,Manuál,70 kW,Benzín
4,Skoda Scala 110 kW,Scala,572 490 Kč,Německo,9 999 km,3/2021,Manuál,110 kW,Benzín
...,...,...,...,...,...,...,...,...,...
195,Skoda Karoq 1.0 TSI 85 kW,Karoq,438 990 Kč,Švédsko,82 000 km,9/2018,Manuál,85 kw,Benzín
196,Skoda Kodiaq 2.0 TSI ACT 4x4 DSG 140 kW,Kodiaq,965 990 Kč,Německo,28 154 km,3/2021,Automat,140 kW,Benzín
197,Skoda Kodiaq 147 kW,Kodiaq,1 086 990 Kč,Německo,26 500 km,7/2021,Automat,147 kW,Diesel
198,Skoda Karoq 1.5 TSI ACT DSG 110 kW,Karoq,765 490 Kč,Německo,25 319 km,1/2021,Automat,110 kW,Benzín


In [56]:
dataframe['price'] = [sub.split('Kč')[0] for sub in dataframe['price']]
dataframe['price'] = [int(sub.replace('\xa0','')) for sub in dataframe['price']]
dataframe['price']

0       986990
1       572490
2      1033990
3       286990
4       572490
        ...   
195     438990
196     965990
197    1086990
198     765490
199     223490
Name: price, Length: 200, dtype: int64

In [57]:
dataframe['mileage'] = [sub.split('km')[0] for sub in dataframe['mileage']]
dataframe['mileage'] = [int(sub.replace('\xa0','')) for sub in dataframe['mileage']]
dataframe['mileage']

0       2000
1      30616
2      27600
3      25147
4       9999
       ...  
195    82000
196    28154
197    26500
198    25319
199    51000
Name: mileage, Length: 200, dtype: int64

In [58]:
dataframe['registration'] = [int(sub.split('/')[1]) for sub in dataframe['registration']]
dataframe['registration']

0      2021
1      2020
2      2021
3      2018
4      2021
       ... 
195    2018
196    2021
197    2021
198    2021
199    2019
Name: registration, Length: 200, dtype: int64

In [59]:
dataframe['power'] = [int(sub.split(' ')[0]) for sub in dataframe['power']]
dataframe['power']

0      132
1      110
2      140
3       70
4      110
      ... 
195     85
196    140
197    147
198    110
199     44
Name: power, Length: 200, dtype: int64

In [65]:
dataframe['model'].unique()

array(['Enyaq iV', 'Scala', 'Kodiaq', 'Rapid', 'Octavia', 'Karoq',
       'Superb', 'Yeti', 'Fabia', 'Citigo', 'Kamiq'], dtype=object)

In [68]:
sum(dataframe['model'] == 'Enyaq iV')

13

In [69]:
sum(dataframe['fueltype'] == 'Elektřina')

13

In [60]:
dataframe['transmission'].unique()

array(['Automat', 'Manuál'], dtype=object)

In [61]:
transmission_dummies = pd.get_dummies(dataframe['transmission'])
dataframe = pd.concat([dataframe,transmission_dummies['Automat']], axis=1)

In [62]:
dataframe['fueltype'].unique()

array(['Elektřina', 'Benzín', 'Diesel', 'LPG', 'Hybrid'], dtype=object)

In [63]:
fueltype_dummies = pd.get_dummies(dataframe['fueltype'])
dataframe = pd.concat([dataframe,fueltype_dummies], axis=1)

In [ ]:
dataframe.drop(['transmission','fuel_type','Benzín'], inplace=True, axis=1)

In [70]:
dataframe

,name,model,price,country,mileage,registration,transmission,power,fueltype,Automat,Benzín,Diesel,Elektřina,Hybrid,LPG
0,Skoda Enyaq iV 60 132 kW,Enyaq iV,986990,Francie,2000,2021,Automat,132,Elektřina,1,0,0,1,0,0
1,Skoda Scala Style 110 kW,Scala,572490,Německo,30616,2020,Manuál,110,Benzín,0,1,0,0,0,0
2,Skoda Kodiaq 2.0 TSI ACT 4x4 DSG 140 kW,Kodiaq,1033990,Německo,27600,2021,Automat,140,Benzín,1,1,0,0,0,0
3,Skoda Rapid 70 kW,Rapid,286990,Polsko,25147,2018,Manuál,70,Benzín,0,1,0,0,0,0
4,Skoda Scala 110 kW,Scala,572490,Německo,9999,2021,Manuál,110,Benzín,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Skoda Karoq 1.0 TSI 85 kW,Karoq,438990,Švédsko,82000,2018,Manuál,85,Benzín,0,1,0,0,0,0
196,Skoda Kodiaq 2.0 TSI ACT 4x4 DSG 140 kW,Kodiaq,965990,Německo,28154,2021,Automat,140,Benzín,1,1,0,0,0,0
197,Skoda Kodiaq 147 kW,Kodiaq,1086990,Německo,26500,2021,Automat,147,Diesel,1,0,1,0,0,0
198,Skoda Karoq 1.5 TSI ACT DSG 110 kW,Karoq,765490,Německo,25319,2021,Automat,110,Benzín,1,1,0,0,0,0


In [72]:
dataframe['country'].unique()

array(['Francie', 'Německo', 'Polsko', 'Slovensko', 'Švédsko', 'Itálie',
       'Španělsko', 'Rakousko', 'Lucembursko', 'Belgie'], dtype=object)

In [74]:

cars_fr = dataframe.loc[dataframe['country'] == 'Francie']
cars_fr

,name,model,price,country,mileage,registration,transmission,power,fueltype,Automat,Benzín,Diesel,Elektřina,Hybrid,LPG
0,Skoda Enyaq iV 60 132 kW,Enyaq iV,986990,Francie,2000,2021,Automat,132,Elektřina,1,0,0,1,0,0
17,Skoda Enyaq iV 80 150 kW,Enyaq iV,1258990,Francie,5000,2021,Automat,150,Elektřina,1,0,0,1,0,0
18,Skoda Fabia 1.0 TSI 70 kW,Fabia,372490,Francie,8185,2021,Manuál,70,Benzín,0,1,0,0,0,0
23,Skoda Citigo 1.0 MPI 44 kW,Citigo,197490,Francie,39980,2017,Manuál,44,Benzín,0,1,0,0,0,0
24,Skoda Karoq 1.0 TSI Ambition 85 kW,Karoq,460990,Francie,60485,2018,Manuál,85,Benzín,0,1,0,0,0,0
36,Skoda Fabia 1.0 TSI Clever 70 kW,Fabia,238490,Francie,73210,2018,Manuál,70,Benzín,0,1,0,0,0,0
40,Skoda Citigo 1.0 MPI Tec 44 kW,Citigo,250490,Francie,15347,2019,Manuál,44,Benzín,0,1,0,0,0,0
42,Skoda Superb Combi 2.0 TDI Green Tec Business ...,Superb,764990,Francie,13850,2021,Automat,110,Diesel,1,0,1,0,0,0
44,Skoda Fabia 1.0 TSI 70 kW,Fabia,410490,Francie,13786,2021,Automat,70,Benzín,1,1,0,0,0,0
48,Skoda Kodiaq 2.0 TDI DSG 110 kW,Kodiaq,675990,Francie,41804,2018,Automat,110,Diesel,1,0,1,0,0,0


In [78]:
def simplemodel(df):
    result = sm.OLS(df['price'], sm.add_constant(df[['mileage','registration','power','Automat','Diesel','Elektřina','Hybrid','LPG']])).fit()
    return result.summary()

In [79]:

simplemodel(cars_fr)    

C:\Users\petrd\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.961
Model:                            OLS   Adj. R-squared:                  0.954
Method:                 Least Squares   F-statistic:                     146.2
Date:                Sat, 05 Feb 2022   Prob (F-statistic):           2.18e-27
Time:                        23:55:31   Log-Likelihood:                -626.32
No. Observations:                  50   AIC:                             1269.
Df Residuals:                      42   BIC:                             1284.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -3.193e+07   2.24e+07     -1.425      0.161   -7.71e+07    1.33e+07
mileage         -1.1100      0.668     -1.660      0.104      -2.459       0.239
registration   1.58e+04   1.11e+04      1.425      0.162   -6580.352    3.82e+04
power         6006.5401    678.173      8.857      0.000    4637.931    7375.150
Automat       5.513e+04   3.93e+04      1.403      0.168   -2.41e+04    1.34e+05
Diesel       -3603.7004   3.41e+04     -0.106      0.916   -7.24e+04    6.51e+04
Elektřina     1.973e+05   4.26e+04      4.630      0.000    1.11e+05    2.83e+05
Hybrid       -1.885e+05   8.42e+04     -2.239      0.031   -3.58e+05   -1.86e+04
LPG                   0          0        nan        nan           0           0
==============================================================================
Omnibus:                        0.153   Durbin-Watson:                   1.893
Prob(Omnibus):                  0.926   Jarque-Bera (JB):                0.360
Skew:                           0.014   Prob(JB):                        0.835
Kurtosis:                       2.585   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [80]:

cars_de = dataframe.loc[dataframe['country'] == 'Německo']
simplemodel(cars_de)


C:\Users\petrd\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     231.2
Date:                Sat, 05 Feb 2022   Prob (F-statistic):           6.79e-52
Time:                        23:55:35   Log-Likelihood:                -1298.0
No. Observations:                 102   AIC:                             2608.
Df Residuals:                      96   BIC:                             2624.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.015e+08   1.58e+07     -6.403      0.000   -1.33e+08      -7e+07
mileage         -0.6213      0.504     -1.232      0.221      -1.623       0.380
registration  5.036e+04   7841.369      6.422      0.000    3.48e+04    6.59e+04
power         3372.5722    462.152      7.298      0.000    2455.208    4289.936
Automat        1.87e+05   2.63e+04      7.101      0.000    1.35e+05    2.39e+05
Diesel        8.512e+04   2.13e+04      4.004      0.000    4.29e+04    1.27e+05
Elektřina             0          0        nan        nan           0           0
Hybrid                0          0        nan        nan           0           0
LPG                   0          0        nan        nan           0           0
==============================================================================
Omnibus:                        4.734   Durbin-Watson:                   2.141
Prob(Omnibus):                  0.094   Jarque-Bera (JB):                4.387
Skew:                          -0.506   Prob(JB):                        0.112
Kurtosis:                       3.085   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [83]:
cars_pl = dataframe.loc[dataframe['country'] == 'Polsko']
simplemodel(cars_pl)

C:\Users\petrd\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     233.0
Date:                Sat, 05 Feb 2022   Prob (F-statistic):           4.59e-11
Time:                        23:56:28   Log-Likelihood:                -206.54
No. Observations:                  19   AIC:                             429.1
Df Residuals:                      11   BIC:                             436.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -5.084e+07   5.74e+06     -8.860      0.000   -6.35e+07   -3.82e+07
mileage         -0.7594      0.118     -6.429      0.000      -1.019      -0.499
registration  2.526e+04   2845.218      8.877      0.000     1.9e+04    3.15e+04
power         2731.2870    249.606     10.942      0.000    2181.908    3280.666
Automat        8.74e+04      2e+04      4.360      0.001    4.33e+04    1.32e+05
Diesel        1.898e+04   1.08e+04      1.754      0.107   -4839.248    4.28e+04
Elektřina    -1.388e-11   1.82e-12     -7.630      0.000   -1.79e-11   -9.88e-12
Hybrid        1.882e+05   2.52e+04      7.471      0.000    1.33e+05    2.44e+05
LPG           1.367e+04   2.05e+04      0.666      0.519   -3.15e+04    5.88e+04
==============================================================================
Omnibus:                        0.702   Durbin-Watson:                   1.581
Prob(Omnibus):                  0.704   Jarque-Bera (JB):                0.710
Skew:                           0.242   Prob(JB):                        0.701
Kurtosis:                       2.186   Cond. No.                     3.94e+22
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.1e-34. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""